In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.6 MB/s eta 0:00:00


Import the necessary packages, as well as install any

In [2]:
# import packages
import pandas as pd
import numpy as np
from transformers import pipeline
import torch
from sklearn.metrics import matthews_corrcoef, f1_score, accuracy_score

Read in the data, make the appropriate labels

In [6]:
from google.colab import files
uploaded = files.upload()
import io
df2 = pd.read_csv(io.BytesIO(uploaded['gtdlabeled.csv']))

Saving gtdlabeled.csv to gtdlabeled (2).csv


In [7]:
# set labels and template
samples = list(df2['summary'])
template = 'This paragraph is describing {} Terrorism.'
labels = ['anti-abortion', 'christian nationalism', 'right wing', 'left wing', 'islamic', 'white-nationalist', 'anti-immigration', 'animal rights', 'environmental']

Create the Zero-shot Classification Model

In [8]:
# create the model, small batch size
classifier = pipeline("zero-shot-classification", model='MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli', device = 0, batch_size = 8)

Classify the data using our model, convert the labels to binary variable for use

In [9]:
# start the classification once model and labels are all set up
res = classifier(samples, labels, hypothesis_template = template, multi_label = False)
df2['label'] = [label['labels'][0] for label in res] # adds most probable label to the dataframe
# convert the text labels to 1 for right-wing and 0 for not
df2['label'].replace(regex = {r'anti-abortion': 1, r'christian nationalism': 1, r'right wing': 1, r'left wing':0, r'islamic':0, r'white-nationalist':1, r'anti-immigration':1, r'animal rights':0, r'environmental':0}, inplace = True)


In [10]:
# examine the accuracy of the classifier
# extract the hand-labeled values and predicted labels
hand_labeled = df2.iloc[np.concatenate([np.arange(50), np.arange(1099, 1150), np.arange(1399, 1450), np.arange(1699, 1750)]), :]['X'].values
predicted_labels = df2.iloc[np.concatenate([np.arange(50), np.arange(1099, 1150), np.arange(1399, 1450), np.arange(1699, 1750)]), :]['label'].values

# calculate the accuracy
accuracy = (hand_labeled == predicted_labels).sum() / len(hand_labeled)
print("Accuracy of zero-shot classifier:", accuracy)


Accuracy of zero-shot classifier: 0.7339901477832512


In [ ]:
df2.to_csv('gtdlabeled.csv', encoding = 'utf-8-sig') 
files.download('gtdlabeled.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>